In [1]:
import bospy.bos as bos
import bospy.run as run
import time

# host = "localhost"
host = "nuc"

bos.SYSMOD_ADDR = "{}:2821".format(host)
bos.DEVCTRL_ADDR = "{}:2822".format(host)
bos.HISTORY_ADDR = "{}:2823".format(host)
run.SCHEDULER_ADDR = "{}:2824".format(host)

### Trying to run a container

In [15]:
r = run.Run('echo', "1", "1.1", "one", key="value", envVars={"KEY": "VALUE"})
r

image: echo
args: ('1', '1.1', 'one')
envVars: {'KEY': 'VALUE'}
kwargs: {'key': 'value'}


StdOut: "\001\000\000\000\000\000\000\031Container received ARGS:\n\001\000\000\000\000\000\000\024[\'1\', \'1.1\', \'one\']\n\001\000\000\000\000\000\000\033Container received KWARGS:\n\001\000\000\000\000\000\000\014key = value\n\001\000\000\000\000\000\000\016demo app done\n"

In [14]:
r = run.Run('random-get')

image: random-get
args: ()
envVars: None
kwargs: {}


_InactiveRpcError: <_InactiveRpcError of RPC that terminated with:
	status = StatusCode.INTERNAL
	details = "grpc: error while marshaling: string field contains invalid UTF-8"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2025-03-07T09:17:42.884467-05:00", grpc_status:13, grpc_message:"grpc: error while marshaling: string field contains invalid UTF-8"}"
>

### Make a new Driver

In [ ]:
# driver = bos.MakeDriver('bacnet', 'bacnet', 50062)
# print(driver)

### Make a new device

In [ ]:
# multisensor = bos.MakeDevice('delta-multisensor')
# multisensor
multisensor = 'bos://localhost/dev/8'

### Create child points
read points from list and attach to newly created device

In [ ]:
import pandas as pd
path = '/Users/jamescoleman/code/bos/services/device/drivers/bacnet/templates/delta_03_sensor.csv'
point_list = pd.read_csv(path)
point_list = point_list[point_list['import'].str.lower() == 'y']
point_list

In [ ]:
# points:list[str] = []
# for i in point_list.index:
#     row = point_list.loc[i]
#     xref = "bacnet://{}/{}/{},{}/{}".format(
#         row['ip_addr'],
#         row['bacnet_instance'],
#         row['object_type'],
#         row['object_number'],
#         row['property'],
#     )
#     print(row['bos_name'], row['brick_type'])
#     print(xref)
#     pt = bos.MakePoint(
#         name=row['bos_name'],
#         device=multisensor,
#         types=row['brick_type'],
#         xref=xref)
#     points.append(pt)
#     print("added {} ({})".format(row['bos_name'], pt))

# points

In [ ]:
dev = bos.QueryDevices()
print("there are {} devices in the system model".format(len(dev)))
print("the first {} pointers are:".format(3))
for p in dev[:3]:
    print("\t", p)

In [ ]:
# bos.Delete(sub="bos://localhost/dev/8")
# bos.Delete(sub="bos://localhost/dev/8/pts/1")

In [ ]:
# new_dev = bos.MakeDevice(name='dummy-device')
# print(new_dev)

In [ ]:
# new_dev = "bos://localhost/dev/8"
# new_pt = bos.MakePoint(name="first-point", device=new_dev, types="brick:On_Command")
# print(new_pt)

In [ ]:
names:dict = {}
for p in points:
    name = bos.GetName(p)
    names[p] = name
    print(name, p)

In [ ]:
light_level = bos.QueryPoints(names='o3_light_level')
light_level

In [ ]:
# bos.Get(light_level)
results = bos.Get(points[:5])
for key in results:
    print(names[key], results[key])

In [ ]:
red_on = bos.QueryPoints(names='red-on')
bos.Set(red_on, False)

In [ ]:
bos.Get(bos.QueryPoints(names='co2'))

In [ ]:
bos.QueryPoints()

In [ ]:
fan_dev = bos.QueryDevices(names='box-fan', types='brick:Fan')
print(fan_dev)

In [ ]:
for d in dev:
    name = bos.GetName(d)
    print(d, name)

In [ ]:
on_pts = bos.QueryPoints(types='brick:On_Command')
print(on_pts)

In [ ]:
bos.Set(on_pts, False)

In [ ]:
power_pts = bos.QueryPoints(types='brick:Power_Sensor')
print(power_pts)

In [ ]:
bos.Get(power_pts)

In [ ]:
pts = bos.QueryPoints()
print("there are {} control points in the system model".format(len(pts)))
print("the first {} pointers are:".format(3))
for p in pts[:3]:
    print("\t", p)

In [ ]:
bos.QueryDevices(names='box-fan')

In [ ]:
named_pts = bos.QueryPoints(names=['yellow-on', 'red-on'])
print("There are {} points named 'yellow-on' or 'red-on'".format(len(named_pts)))
print(named_pts)

In [ ]:
on_pts = bos.QueryPoints(types='brick:On_Command')
print("There are {} brick:On_Command typed points".format(len(on_pts)))
print(on_pts)

In [ ]:
on_pts = bos.QueryPoints(types='brick:On_Command', parent_types='brick:Luminaire')
print("There are {} brick:On_Command typed points that belong to a brick:Luminaire ".format(len(on_pts)))
print(on_pts)

In [ ]:
statuses = bos.Get(on_pts)
print(statuses)

In [ ]:
pts = bos.QueryPoints(names=['fan-power', 'main-power'])
print(pts)

In [ ]:
bos.SetSampleRate(pts, '1m')


In [ ]:
for p in on_pts:
    name = bos.PointName(p)
    print(name, "=", statuses[p])
    

In [ ]:
fan_on = bos.QueryPoints(names='fan-on')
print(fan_on)

In [ ]:
bos.Set(fan_on, False)

In [ ]:
bos.Set(on_pts, [True, True]) # turn on all lights
time.sleep(5)                  
bos.Set(on_pts, False)        # turn off all lights

In [ ]:
temp_pts = bos.QueryPoints(locations='lab', types='brick:Air_Temperature_Sensor')
temps = bos.Get(temp_pts)
for p in temp_pts:
    name = bos.PointName(p)
    print(name, "=", temps[p])

mean_temp = sum(temps.values()) / len(temps)
print("===")
print("mean_lab_temp = {:.1f}".format(mean_temp) )

In [ ]:
elec = bos.QueryPoints(parent_types="brick:Electrical_Meter",)
elec = elec[-4:]
elec

In [ ]:
fan_pts = bos.QueryPoints(parent_types='brick:Fan')

In [ ]:
for name in [bos.PointName(pt) for pt in fan_pts]:
    print(name)

In [ ]:
fan_power = bos.QueryPoints(names='fan-power')
print(fan_power)
bos.Get(fan_power)


In [ ]:
fan_on = bos.QueryPoints(names='fan-on')
print(fan_on)
bos.Get(fan_on)

In [ ]:
bos.Set(fan_on, False)

In [ ]:
results = bos.Get(elec)
for k, v in results.items():
    print(k,v)

In [ ]:

fan_on = bos.QueryPoints(types='brick:On_Command', parent_types='brick:Fan')
print(fan_on)
bos.Set(fan_on, False)












In [ ]:
df = bos.GetHistory(temp_pts,
                    start='2025-01-27 00:00:00-05',
                    end=  '2025-01-28 00:00:00-05', tz='US/Eastern',
                    pandas=True, resample_to='5min', get_names=True)
df

In [ ]:
df.plot()

In [ ]:
for p, v in values.items():
    name = bos.PointName(p)
    print(name, "=", v)

In [ ]:
bos.Set(named_pts, False)

In [ ]:
bos.Get(named_pts)

In [ ]:

pt_name = bos.PointToName(pts[0])
print('"{}" is accessed via <{}>'.format(pt_name, pts[0]))

In [ ]:
pts = bos.QueryPoints(
    parent_types='brick:Thermostat')
print(len(pts))
pts

In [ ]:
pts = bos.QueryPoints(
    types='brick:On_Command',
    parent_types='brick:Luminaire')
print(len(pts))
pts

In [ ]:
pts = bos.QueryPoints(
    types='brick:On_Command',
    parent_types='brick:Fan')
print(len(pts))
pts

In [ ]:
bos.Set(pts, False)

In [ ]:
pts = bos.QueryPoints(
    types='brick:Power_Sensor',
    parent_types='brick:Luminaire')
print(len(pts))
pts

In [ ]:
powerPts = bos.QueryPoints(
    types='brick:Power_Sensor')
powerPts

In [ ]:
all_on_pts = bos.QueryPoints(types='brick:On_Command')
resp = bos.Set(all_on_pts, False)

In [ ]:
resp = bos.Set(all_on_pts, True)
print(resp)
time.sleep(5)
values = bos.Get(powerPts)
for k, v in values.items():
    print(k,v )

In [ ]:

names = {pt: bos.PointToName(pt) for pt in powerPts}
for pt in powerPts:
    print("{} = {}".format(names[pt], values[pt]))

In [ ]:

bos.Set(all_on_pts, False)

In [ ]:
bos.Get(pts)

In [ ]:
pts = bos.QueryPoints()
pts = sorted(pts)
for i, point in enumerate(pts):
    name = bos.PointToName(point)
    value = bos.Get(point)
    print("{} = {}".format(point, name))

In [ ]:
pts = bos.QueryPoints()
pts = sorted(pts)
for i, point in enumerate(pts):
    name = bos.PointToName(point)
    value = bos.Get(point)
    print("({}) {} = {}".format(point, name, value[point]))

In [ ]:
pts = bos.QueryPoints()
print(len(pts))

In [ ]:
bos.SetSampleRate(pts, '5m')

In [ ]:
onPt = bos.NameToPoint('on')
bos.Set(onPt, False)

In [ ]:
power = bos.NameToPoint('power')
V = bos.Get(power)
print("using {} watts".format(V[power]))
bos.Set(onPt, False)

In [ ]:
names = [(p, bos.PointToName(p)) for p in pts]
for n in names:
    print(n)

In [ ]:
bos.Set(onPt, False)

In [ ]:
powerPt = bos.NameToPoint("power")
bos.Get(powerPt)

In [ ]:
air_temp_1 = bos.NameToPoint("air_temp")
air_temp_2 = bos.NameToPoint("air_temp_east")
air_temp_3 = bos.NameToPoint("air_temp_west")
air_temps = [air_temp_1, air_temp_2, air_temp_3]
print(air_temps)

In [ ]:
air_temps = bos.QueryPoints(types='brick:Air_Temperature_Sensor',
                            parent_types='brick:Thermostat')
print(air_temps)

In [ ]:
df = bos.GetHistory(air_temps,
                    start='2025-01-26 00:00:00-05',
                    end='2025-01-27 00:00:00-05', tz='US/Eastern',
                    pandas=True, resample_to='5min', get_names=True)
df

In [ ]:
df.plot()

In [ ]:
import numpy as np
# TODO get all the power readings
power_vector = np.array(list(power_readings.values()))

avg_power = power_vector.mean()
total_power = power_vector.sum()

print("the total power draw is {:.1f} W and the average of {} devices is {:.1f} W".format(
    total_power, len(power_vector), avg_power
))

In [ ]:
bos.GetHistory(pts)

In [ ]:
bos.Set(pts, False)

In [ ]:
pt = bos.NameToPoint('air_temp_setpoint')
v = bos.Get(pt)
print(pt, v)

In [ ]:
import time
# get the point named "on", then turn it on
onPt = bos.NameToPoint('on')
ok = bos.Set(onPt, True)

# what 4 seconds for the power measurement to be accurate
time.sleep(4)

# get the point named power, then read it
powerPt = bos.NameToPoint('power')
power_draw = bos.Get(powerPt)[powerPt]

# set the point to off
onPt = bos.Set(onPt, False)
print(ok)

print("While on the bulb draws {} watts".format(power_draw))

In [ ]:
v = bos.Get(pt)
v[pt]